# 模組 4.4: 高基數特徵處理 (Handling High Cardinality)

## 學習目標
- 理解什麼是高基數 (High Cardinality) 特徵以及它們帶來的挑戰。
- 比較不同編碼方法在高基數場景下的適用性。
- 學習一種處理高基數特徵的實用策略：特徵哈希 (Feature Hashing)。
- 討論結合業務邏輯進行特徵合併或簡化的思路。

## 導論：當類別太多時

**基數 (Cardinality)** 是指一個特徵中唯一值的數量。**高基數** 特徵是指那些擁有大量唯一值的類別特徵，例如：用戶 ID、郵遞區號、街道名稱、商品編號等。

這些特徵帶來了獨特的挑戰：
- **獨熱編碼 (One-Hot Encoding)** 會導致「維度災難」，產生數千甚至數萬個新欄位，使模型難以訓練且容易過擬合。
- **標籤編碼 (Label Encoding)** 會引入大量無意義的順序關係。
- **計數/頻率編碼** 可能會因為大量類別只出現一兩次而產生許多相同的編碼值，失去區分度。

因此，我們需要一些特殊的策略來應對它們。

In [2]:
# 導入必要的函式庫
import pandas as pd
from sklearn.feature_extraction import FeatureHasher



## 1. 準備高基數資料

我們手動創建一個模擬的資料集，其中 'ZipCode' 是一個高基數特徵。

In [3]:
# 創建一個包含高基數特徵的 DataFrame
data = {
    'TransactionID': range(10),
    'ZipCode': ['10001', '90210', '10001', '60601', '94105', '90210', '75001', '60601', '10001', '94105'],
    'Amount': [100, 250, 80, 120, 300, 50, 200, 150, 90, 400]
}
df = pd.DataFrame(data)

# 查看 'ZipCode' 的基數
cardinality = df['ZipCode'].nunique()
print(f"原始 DataFrame (ZipCode 的基數為: {cardinality}):")
display(df)




原始 DataFrame (ZipCode 的基數為: 5):


,TransactionID,ZipCode,Amount
0,0,10001,100
1,1,90210,250
2,2,10001,80
3,3,60601,120
4,4,94105,300
5,5,90210,50
6,6,75001,200
7,7,60601,150
8,8,10001,90
9,9,94105,400


## 2. 回顧現有方法的局限性

- **獨熱編碼**: 對這個例子來說，尚可接受。但如果 `ZipCode` 有 10000 個唯一值，就會產生 10000 個新欄位。
- **計數編碼**: `10001` (3次), `90210` (2次), `60601` (2次), `94105` (2次), `75001` (1次)。注意，`90210`, `60601`, `94105` 會被編碼為相同的值 (2)，模型將無法區分它們。

顯然，我們需要更有效的方法。

## 3. 特徵哈希 (Feature Hashing / The Hashing Trick)

**原理**: 特徵哈希是一種快速、低內存消耗的特徵向量化技術。它不建立一個類別到索引的映射表，而是使用一個 **哈希函數 (Hash Function)** 將類別名稱（字串）直接轉換為一個固定範圍內的整數（即新特徵的索引）。

- **哈希函數**: 一個能將任意長度的輸入，通過一個算法，轉換為一個固定長度輸出的函數。
- **哈希衝突 (Hash Collision)**: 因為輸出空間是固定的，不同的輸入（類別）有可能被哈希到同一個輸出索引上。這是特徵哈希的主要缺點，但實踐中，如果哈希空間（`n_features`）設置得當，影響通常是可控的。

In [4]:
# 使用 scikit-learn 的 FeatureHasher
# n_features 決定了輸出特徵的維度（哈希空間的大小）
# 我們將其設置為 4，小於基數 5，以觀察哈希衝突
hashing_encoder = FeatureHasher(n_features=4, input_type='string')

# FeatureHasher 需要一個 list of lists 作為輸入
hashed_features = hashing_encoder.fit_transform(df['ZipCode'].apply(lambda x: [x]))

# 將稀疏矩陣結果轉換為 DataFrame
hashed_df = pd.DataFrame(hashed_features.toarray(), columns=[f'ZipHash_{i}' for i in range(4)])

# 合併結果
df_hashed = pd.concat([df, hashed_df], axis=1)

print("特徵哈希後的 DataFrame (n_features=4):")
display(df_hashed)




特徵哈希後的 DataFrame (n_features=4):


,TransactionID,ZipCode,Amount,ZipHash_0,ZipHash_1,ZipHash_2,ZipHash_3
0,0,10001,100,0.0,0.0,0.0,-1.0
1,1,90210,250,0.0,0.0,1.0,0.0
2,2,10001,80,0.0,0.0,0.0,-1.0
3,3,60601,120,-1.0,0.0,0.0,0.0
4,4,94105,300,0.0,0.0,-1.0,0.0
5,5,90210,50,0.0,0.0,1.0,0.0
6,6,75001,200,-1.0,0.0,0.0,0.0
7,7,60601,150,-1.0,0.0,0.0,0.0
8,8,10001,90,0.0,0.0,0.0,-1.0
9,9,94105,400,0.0,0.0,-1.0,0.0


**解讀**:
- 每個郵遞區號都被轉換成了一個 4 維的向量。
- 哈希值通常是 1 或 -1，這有助於在線性模型中保持權重的平衡。
- 注意，我們沒有儲存任何映射字典，整個過程是無狀態的 (stateless)，這使得它非常適合處理大型或流式資料。
- 因為我們設置的 `n_features=4` 小於基數 5，哈希衝突必然發生，但我們無法直接看出是哪些類別衝突了。

**優點**:
- **速度快，內存效率高**: 無需維護一個龐大的詞典。
- **可擴展性強**: 非常適合處理線上學習 (Online Learning) 或流式資料，因為它不需要提前看到所有類別。

**缺點**:
- **資訊損失與衝突**: 哈希是單向的，無法從哈希值反推回原始類別。哈希衝突會導致模型無法區分不同的原始類別。
- **可解釋性差**: 生成的特徵沒有直觀的業務含義。

## 4. 結合業務邏輯的處理思路

除了純粹的技術方法，處理高基數特徵的另一個重要思路是 **降維 (Dimensionality Reduction)**，但這裡的降維是基於業務理解的。

- **特徵合併**:
  - **基於地理位置**: 將郵遞區號 `ZipCode` 根據其所屬的州 (State) 或城市 (City) 進行分組。例如，所有紐約市的郵編都可以被編碼為 "NewYorkCity"。
  - **基於頻率**: 將所有出現次數小於某個閾值（例如，10次）的稀有類別，全部合併為一個新的類別，如 "Other"。
- **特徵拆分**:
  - 從一個複雜的 ID 中提取有意義的部分。例如，從商品編號 "FR-TECH-1001" 中，可以拆分出 "FR" (國家), "TECH" (品類) 等新特徵。

這種基於業務邏輯的方法通常比純技術方法更有效， çünkü它保留了特徵的內在含義。

## 總結

處理高基數特徵沒有萬能的解決方案，通常需要權衡和實驗。

- 如果**模型性能**是首要目標，且對可解釋性要求不高，**目標編碼**（帶有交叉驗證和隨機性）和 **特徵哈希** 是強有力的選項。
- 如果**可解釋性**很重要，或者你有一定的業務理解，**基於業務邏輯的特徵合併/拆分** 通常是最佳起點。
- 對於樹模型，**計數/頻率編碼** 也可以作為一個簡單有效的基線方法。

在實戰中，常常會結合多種方法，例如先將稀有類別合併為 "Other"，然後再對剩餘的類別進行獨熱編碼或目標編碼。